In [ ]:
#Import required linraries
import os
from pydriller import RepositoryMining
from radon.raw import analyze
import pandas as pd

In [21]:
filepath = 'python_dataset.csv'

df_main = pd.read_csv(filepath, index_col=None, usecols = ['CommitID','filename','complexity','token_count',
                                                           'loc','lloc','sloc','comments',
                                                           'multi','blank','code_comment','h1','h2','N1','N2',
                                                           'vocabulary','length','calculated_length','volume', 
                                                           'difficulty','effort','time','bugs','defect'])

In [ ]:
#Define lists to append values to
hash_val =[]
change_type = []
removed = []
added = []

for commit in RepositoryMining(#path_to_repo#,
                                   only_modifications_with_file_types=['.py'] ).traverse_commits():
        for modification in commit.modifications:
            filename = modification.filename
            if filename.endswith(".py"):
                for r, d, f in os.walk(repo_updated):
                    for file in f:
                        if filename in file:
                            file_path = os.path.join(r, file)
                            with open(file_path) as f:
                                content = f.read()
                                file_analyze = analyze(content)
                                code_n_comment = file_analyze.loc+file_analyze.comments
                                #Append code churn attributes to the lists
                                hash_val.append(commit.hash)
                                change_type.append(modification.change_type)
                                added.append(modification.added)
                                removed.append(modification.removed)
                                

In [66]:
#Add new columns to the main dataset
df_new = pd.DataFrame(columns=['hash_val','change_type','added','removed'])

#write labelled data to CSV with new features
df_main.to_csv('python_02_main.csv')

In [ ]:
#fetch the same file and add headers
filepath = 'python_02_main.csv'
column = ['complexity','token_count','loc','lloc','sloc','comments',
          'multi','blank','code_comment','h1','h2','N1','N2','vocabulary','length',
          'calculated_length','volume', 'difficulty','effort','time','bugs','change_type','added','removed',
          'defect']
df1 = pd.read_csv(filepath, usecols= column, index_col=False)

In [ ]:
#Pre-processing using ordinal encoder
categories =['ModificationType.ADD', 'ModificationType.DELETE', 
            'ModificationType.MODIFY','ModificationType.RENAME']
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df1.change_type = encoder.fit_transform(df1.change_type.values.reshape(-1, 1))

#Clean data by removing rows where LOC = 0, and complexity = 0
col = ['loc','complexity']
df1 = df1.replace(0, pd.np.nan).dropna(axis=0,how='any',subset=col).fillna(0).astype(int)

#update data to CSV
df1.to_csv('python_02.csv')